## Register User

In [1]:
!git clone https://github.com/Mahsarnzh/Pass_Cracker_Tool.git

fatal: destination path 'Pass_Cracker_Tool' already exists and is not an empty directory.


In [2]:
%cd /content/Pass_Cracker_Tool

/content/Pass_Cracker_Tool


In [3]:
def read_wordlist(file_path):
    try:
        with open(file_path, 'r') as wordlist_file:
            wordlist = [word.strip() for word in wordlist_file.readlines() if len(word.strip()) <= 100]
        return wordlist
    except FileNotFoundError:
        print(f"Error: Wordlist file '{file_path}' not found.")
        exit()
    except Exception as e:
        print("Error while reading the wordlist file:", e)
        return []

wordlist = read_wordlist("dictionary.txt")

In [7]:
%cd /content/Pass_Cracker_Tool

/content/Pass_Cracker_Tool


In [ ]:
import getpass
import hashlib

def register_user():
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")

    # Create the MD5 hash of the password
    password_hash = hashlib.md5(password.encode()).hexdigest()

    # Save username and password hash to a file
    with open("user_info_MD5.txt", "a") as file:
        file.write(f"{username}:{password_hash}\n")

    print("User registered successfully.")

# Call the register_user function to register a new user
register_user()


## Verify User

In [ ]:
import getpass
import hashlib

def verify_user():
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")

    # Create the MD5 hash of the entered password
    password_hash = hashlib.md5(password.encode()).hexdigest()

    # Read the password file and check for username and password hash match
    with open("user_info_MD5.txt", "r") as file:
        for line in file:
            stored_username, stored_password_hash = line.strip().split(":")
            if username == stored_username and password_hash == stored_password_hash:
                print("Login successful!")
                return

    print("Invalid username or password.")

# Call the verify_user function to verify the user's login credentials
verify_user()


## Password Cracker

In [ ]:
import hashlib
import numpy as np
import itertools
from itertools import combinations
from itertools import permutations
from itertools import product


def create_combinations(words):
    words = [words]
    num_arr = list(range(10))
    special_char_arr = ["@","#","$","%","&", "", ""]
    num_arr.extend(["",""])
    int_list = list(combinations(num_arr, 2))

    char_list = list(combinations(special_char_arr, 2))

    words_arr = np.array(words)
    int_arr = np.array(int_list).reshape(-1,2)
    char_arr = np.array(char_list).reshape(-1,2)

    product_ = list(product(int_arr, char_arr))
    product_ = np.array(product_).reshape(-1,4)

    words_comb = list(product(words_arr, product_))
    words_comb_res = [[t[0]] + t[1].tolist() for t in words_comb]

    word_comb_perm_ = []
    for i in range(len(words_comb_res)):
      permutations = list(itertools.permutations(words_comb_res[i]))
      word_comb_perm = [''.join(perm) for perm in permutations]
      word_comb_perm_.append(word_comb_perm)

    word_comb_perm_unique = np.unique(word_comb_perm_)
    return word_comb_perm_unique


def hash_password(password):
    result = hashlib.md5(password.encode())
    return result.hexdigest()

def check_password_in_file(password_hash):
    wordlist = read_wordlist("dictionary.txt")
    for word in wordlist:
        if hash_password(word) == password_hash:
            return word
        else:
          for word in create_combinations(word):
            if hash_password(word) == password_hash:
              return word
    return None

if __name__ == "__main__":

    with open("user_info_MD5.txt", 'r') as md5_file:
        md5_hashes_to_check = md5_file.read().splitlines()
        md5_hashes_data = [item.split(":") for item in md5_hashes_to_check]

    # Reshape the list into a 2x2 matrix (if you have more than two elements, change the shape accordingly)
    md5_items = np.array(md5_hashes_data).reshape(-1, 2)

    matching_passwords = []
    for md5_item in md5_items:
        password_hash = md5_item[1]  # Extract the password hash from the matrix
        found_password = check_password_in_file(password_hash)
        if found_password:
            matching_passwords.append(found_password)

    if matching_passwords:
        print("The following passwords were found in the dictionary file:")
        for password in matching_passwords:
            print(password)
    else:
        print("No passwords were found in the dictionary file.")



## Evaluating Password Strength

In [20]:
import numpy as np
import itertools
from itertools import combinations
from itertools import permutations
from itertools import product


wordlist = read_wordlist("dictionary.txt")


def create_combinations(words):
    words = [words]
    num_arr = list(range(10))
    special_char_arr = ["@","#","$","%","&", "", ""]
    num_arr.extend(["",""])
    int_list = list(combinations(num_arr, 2))

    char_list = list(combinations(special_char_arr, 2))

    words_arr = np.array(words)
    int_arr = np.array(int_list).reshape(-1,2)
    char_arr = np.array(char_list).reshape(-1,2)

    product_ = list(product(int_arr, char_arr))
    product_ = np.array(product_).reshape(-1,4)

    words_comb = list(product(words_arr, product_))
    words_comb_res = [[t[0]] + t[1].tolist() for t in words_comb]

    word_comb_perm_ = []
    for i in range(len(words_comb_res)):
      permutations = list(itertools.permutations(words_comb_res[i]))
      word_comb_perm = [''.join(perm) for perm in permutations]
      word_comb_perm_.append(word_comb_perm)

    word_comb_perm_unique = np.unique(word_comb_perm_)
    return word_comb_perm_unique



def is_weak_password(password, wordlist):
  for word in wordlist:
    if word == password:
      return True
  return False


def is_moderate_password(password, wordlist):
  for word in wordlist:
    for word in create_combinations(word):
      # print(word)
      if word == password:
        return True
  return False


def evaluate_password_strength(password, wordlist):
    """Evaluate password strength based on dictionary attack vulnerability."""
    if is_weak_password(password, wordlist):
        return "Weak password"
    elif is_moderate_password(password, wordlist):
        return "Moderate password"
    else:
        return "Strong password"


if __name__ == "__main__":
    # Load dictionary words from a file (replace 'dictionary.txt' with the actual file path)
    dictionary_file_path = 'dictionary.txt'
    dictionary_words = read_wordlist(dictionary_file_path)

    # Take password input from the user
    password = input("Enter your password: ")

    # Evaluate the password strength
    password_strength = evaluate_password_strength(password, dictionary_words)
    print("Password strength:", password_strength)


Enter your password: authentic#
Password strength: Moderate password
